In [1]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD,rmsprop
import numpy as np
from collections import defaultdict
import operator
from skimage import data, io, filters, transform
import skimage

Using TensorFlow backend.


In [2]:
#read tags_list and sort by count, store in tags_list to be later used with images
tags = defaultdict(int)
tags_list = []
tags_file = open("Harrison/tag_list.txt","r")
count = 0
for line in tags_file:
    tags_list.append(line)
    tag = line.split()
    count += 1
    for i in tag:
        tags[i] += 1
sorted_tags = sorted(tags.items(), key=operator.itemgetter(1), reverse=True)

In [3]:
print len(tags_list)

57383


In [3]:
popular_tags_count = sorted_tags[:20]

In [4]:
#top 20 popular tags
popular_tags = [x[0] for x in popular_tags_count]
print popular_tags

['love', 'friend', 'beach', 'family', 'yellow', 'girl', 'fashion', 'nike', 'snow', 'happy', 'bored', 'school', 'beautiful', 'tired', 'black', 'sun', 'selfie', 'instagood', 'nature', 'smile']


In [26]:
# read image path and add them in dictionary {image_path, tags}, got tags from tags_list
image_tags = {}
image_file = open("Harrison/data_list.txt","r")
count = 0
for line in image_file:
    tags = tags_list[count]
    count += 1
    image_tags[line] = tags.split()

In [30]:
#store only two tags per image
images = {}
for key,value in image_tags.iteritems():
    tags_n = [i for i in value if i in popular_tags]
    if len(tags_n)>0:
        images[key] = tags_n


In [31]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))
n_items = take(1000, images.iteritems())
#print n_items

In [32]:
print len(images)

34514


In [33]:
#encode tags for training
encoded_tags = []
for value in images.values():
    temp = np.zeros(20)
    for i in value:
        temp[popular_tags.index(i)] = 1
    encoded_tags.append(temp)

In [11]:
test = images["instagram_dataset/loveyou/image_2659.jpg\n"]
index1 = popular_tags.index(test[0])
index2 = popular_tags.index(test[1])
#index3 = popular_tags.index(test[2])
print encoded_tags[0]
print encoded_tags[0][index2], encoded_tags[0][index3],encoded_tags[0][index1+10]

[ 0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
0.0

NameError: name 'index3' is not defined

In [34]:
def load_image(path):
    # load image
    img = skimage.io.imread(path)
    img = img / 255.0
    assert (0 <= img).all() and (img <= 1.0).all()
    # print "Original Image Shape: ", img.shape
    # we crop image from center
    short_edge = min(img.shape[:2])
    yy = int((img.shape[0] - short_edge) / 2)
    xx = int((img.shape[1] - short_edge) / 2)
    crop_img = img[yy: yy + short_edge, xx: xx + short_edge]
    # resize to 224, 224
    resized_img = skimage.transform.resize(crop_img, (224, 224,3))
    return resized_img

In [35]:
#read images
image_data = []
count=0
for key in n_items:
    if count%1000==0:
        print count
    count+=1
    im = load_image("Harrison/"+key[0].strip())
  #  im = im.reshape(224,224,3)
 #   temp = im.reshape((1, 224, 224, 3))
    im -= np.mean(im)
   # im = im.transpose((2,0,1))
  #  im = np.expand_dims(im, axis=0)
    image_data.append(im)

0


In [ ]:
print len(encoded_tags)

In [38]:
#sample, train, test
sample = np.array(image_data).astype(np.float32)
sample_tags = encoded_tags[:1000]
x_train = sample[:800]
y_train = sample_tags[:800]
x_test = sample[800:]
y_test = sample_tags[800:]

In [39]:
print np.array(x_train).shape, np.array(y_train).shape, np.array(x_test).shape, np.array(y_test).shape

(800, 224, 224, 3) (800, 20) (200, 224, 224, 3) (200, 20)


In [40]:
#define model layers
def VGG_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224,3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2),data_format="channels_last"))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2),data_format="channels_last"))
    """
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2),data_format="channels_last"))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2),data_format="channels_last"))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512,(3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2),data_format="channels_last"))
    """
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
 #   model.add(Dense(4096, activation='relu'))
  #  model.add(Dropout(0.5))
    model.add(Dense(20, activation='sigmoid'))

    return model

In [41]:
import tensorflow as tf
model = VGG_16()
model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['accuracy'])
with tf.device('cpu:0'):
    model.fit(np.array(x_train),np.array(y_train), epochs=1, batch_size=32)

Epoch 1/1
800/800 [==============================] - 878s 1s/step - loss: 0.4911 - acc: 0.8758

In [42]:
preds = model.predict(np.array(x_test))

In [43]:
#preds[preds>=0.5] = 1
#preds[preds<0.5] = 0
print preds[1]

[ 0.18779427  0.09658569  0.09420606  0.12926207  0.19337751  0.10877515
  0.1129993   0.1081184   0.11774207  0.08429874  0.16873688  0.09957865
  0.16282535  0.08237048  0.12376712  0.07985076  0.09594902  0.12069392
  0.07909167  0.12999049]


In [44]:
print preds[1].argsort()[-2:][::-1]

[4 0]


In [53]:
one = []
two = []
three = []
count_c = defaultdict(int)
count = 0
for i in preds:
    predicted_tags = i.argsort()[-5:][::-1]
    correct_tags = np.nonzero(y_test[count])
   # print predicted_tags, correct_tags[0]
    correct_predicted = [i for i in predicted_tags if i in correct_tags[0]]
    count_c[len(correct_predicted)] += 1
    
    count +=1

In [54]:
print count_c

defaultdict(<type 'int'>, {0: 117, 1: 72, 2: 11})


In [27]:
test = defaultdict(int)
for n in n_items:
    t = n[1]
    for i in t:
        test[i]+=1

In [28]:
print test

defaultdict(<type 'int'>, {'beautiful': 114, 'school': 76, 'fashion': 54, 'love': 286, 'nike': 15, 'family': 127, 'tired': 39, 'sun': 97, 'selfie': 123, 'nature': 73, 'snow': 39, 'yellow': 38, 'instagood': 79, 'black': 54, 'smile': 134, 'girl': 169, 'bored': 48, 'beach': 90, 'friend': 186, 'happy': 159})
